<a href="https://colab.research.google.com/github/skumarudel/DSPS_SKumar/blob/master/HW10/AutoEncoders_OnFaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")